In [1]:
import os
import sys
sys.path.append('../..')
import utils

import panel as pn  # GUI
import ollama  # Using Ollama for LLaMA 3

pn.extension()

In [2]:
def get_completion_from_messages(messages, model="phi3", temperature=0, max_tokens=500):
    """Generate response using Ollama's phi3 model."""
    formatted_messages = [{"role": msg["role"], "content": msg["content"]} for msg in messages]
    
    try:
        response = ollama.chat(model=model, messages=formatted_messages)
        return response['message']['content'].strip()  # Ensure clean output
    except Exception as e:
        print("ERROR: Ollama API call failed:", str(e))
        return "[]"  # Return empty JSON list on failure

In [3]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"

    # Step 1: Input validation
    if debug:
        print("Step 1: Input passed basic check.")

    # Step 2: Extract products and categories
    try:
        category_and_product_response = utils.find_category_and_product_only(
            user_input, utils.get_products_and_category()
        )
        category_and_product_list = utils.read_string_to_list(category_and_product_response)
    except Exception as e:
        category_and_product_list = []
        if debug:
            print(f"Step 2 Error: Failed to extract categories/products - {e}")

    if debug:
        print(f"Step 2: Extracted products: {category_and_product_list}")

    # Step 3: Fetch product information
    product_information = utils.generate_output_string(category_and_product_list) if category_and_product_list else "No matching products found."

    if debug:
        print("Step 3: Retrieved product information.")

    # Step 4: Generate customer service response
    step_4_system_message_content = "You are a helpful AI assistant. Provide detailed and accurate information."

    messages = [
        {"role": "system", "content": step_4_system_message_content},
        {"role": "user", "content": f"{delimiter}{user_input}{delimiter}"},
        {"role": "assistant", "content": f"Relevant product information:\n{product_information}"},
    ]

    all_messages.extend(messages)
    final_response = get_completion_from_messages(all_messages)
    all_messages.append({"role": "assistant", "content": final_response})

    if debug:
        print("Step 4: Generated response.")

    # Step 5: Evaluate response quality
    step_6_system_message_content = "Evaluate whether the AI response fully answers the customer's question."

    evaluation_messages = [
        {"role": "user", "content": step_6_system_message_content +
         f"\n\nCustomer message: {delimiter}{user_input}{delimiter}\n" +
         f"Agent response: {delimiter}{final_response}{delimiter}\n" +
         f"Does the response sufficiently answer the question? Reply in a single word: Y or N"}
    ]

    evaluation_response = get_completion_from_messages(evaluation_messages)
    cleaned_evaluation_response = evaluation_response.strip().upper()

    if debug:
        print(f"Step 6: Evaluation result: {cleaned_evaluation_response}")

    # Step 7: Decision based on evaluation
    if cleaned_evaluation_response == "Y":
        if debug:
            print("Step 7: Response is approved.")
        return final_response, all_messages
    elif cleaned_evaluation_response == "N":
        if debug:
            print("Step 7: Response is not sufficient.")
        return "I'm unable to provide the information you're looking for. Let me connect you with a representative.", all_messages
    else:
        if debug:
            print(f"Step 7: Unexpected evaluation response: {cleaned_evaluation_response}")
        return "There was an issue processing your request. Please try again.", all_messages

# Test the function
user_input = "Tell me about the SmartX Pro Phone and the FotoSnap Camera, the DSLR one. Also, what tell me about your TVs?"
response, _ = process_user_message(user_input, [])
print(response)


Step 1: Input passed basic check.
Raw Response from Ollama: ```json
[
    {
        "category": "Smartphones and Accessories",
        "products": ["SmartX ProPhone"]
    },
    {
        "category": "Cameras and Camcorders",
        "products": ["FotoSnap DSLR Camera"]
    },
    {
        "category": "Televisions and Home Theater Systems",
        "products": ["CineView 4K TV", "CineView OLED TV"]
    }
]
```
Step 2: Extracted products: [{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'CineView OLED TV']}]
Step 3: Retrieved product information.
Step 4: Generated response.
Step 6: Evaluation result: NO, IT DOES NOT ADDRESS ANYTHING ABOUT SMARTX PRO PHONE AND FOTOSNAP CAMERA MODELS NOR THEIR DSLR VERSIONS OR TVS AS REQUESTED BY THE CUSTOMER. IT DISCUSSES COMPLETELY DIFFERENT PRODUCT (CINEVIEW MINI

In [4]:
pn.extension(raw_css=['''
.assistant-response {
    background-color: #F6F6F6;
    padding: 10px;
    border-radius: 5px;
}
'''])

In [5]:
def collect_messages(debug=False):
    user_input = inp.value
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context

    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role': 'assistant', 'content': f"{response}"})

    panels.append(pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600, css_classes=['assistant-response'])))

    return pn.Column(*panels)

panels = []  # collect display
context = [{'role': 'system', 'content': "You are a Service Assistant"}]

inp = pn.widgets.TextInput(placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)